Import the necessary packages

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split

Importing the dataset

In [2]:
dataset = pd.read_csv("C:/Users/Deepu/Desktop/Predict criminal/criminal_train.csv")
X = dataset.iloc[:, 0:71].values
y = dataset.iloc[:, 71].values

Splitting the dataset into the Training set and Test set

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, stratify=y,random_state = 0)

Applying Grid Search to find the best parameters

In [4]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
parametersXGB = [{'max_depth': [2], 'booster': ['gbtree'], 'objective': ['binary:logistic'],
                'gamma':[0.02,0.03],'n_estimators':[100],
                'learning_rate':[0.05,0.06],'colsample_bylevel':[1],'colsample_bytree':[1],'subsample':[1]}]
grid_searchXGB = GridSearchCV(estimator = XGBClassifier(),
                           param_grid = parametersXGB,
                           scoring = 'precision',
                           cv = 15,
                           n_jobs = -1)

 Fitting XGBoost to the Training set

In [5]:
grid_searchXGB = grid_searchXGB.fit(X_train, y_train)

Refit the model if the parameters are changed

In [6]:
grid_searchXGB.refit
y_pred=grid_searchXGB.predict(X_test)
best_accuracyXGB = grid_searchXGB.best_score_ #gives the best accuracy that can be achieved for the testing data
best_parametersXGB = grid_searchXGB.best_params_ #gives the best parameters that can be used from the given list of parameters 

In [7]:
print("Best possible accuracy that can be achieved for testing data :- %.2f%%" %(best_accuracyXGB*100))

Best possible accuracy that can be achieved for testing data :- 70.55%


In [8]:
print("Parameters that are good enough to achieve best precision :-\n ", best_parametersXGB)

Parameters that are good enough to achieve best precision :-
  {'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 0.02, 'learning_rate': 0.06, 'max_depth': 2, 'n_estimators': 100, 'objective': 'binary:logistic', 'subsample': 1}


Calculate the accuracy and the precision score

In [9]:
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
precision=precision_score(y_true=y_test,y_pred=y_pred,average='binary')

In [10]:
print("Accuracy Score: %.2f%%" % (accuracy * 100.0))

Accuracy Score: 95.10%


In [11]:
print("Precison Score: %.2f%%" % (precision * 100.0))

Precison Score: 75.18%


Create the Confusion Matrix

In [12]:
from sklearn.metrics import confusion_matrix
confusion_XGB = confusion_matrix(y_test, y_pred)

In [13]:
print('Confusion matrix for the trained model :-\n',confusion_XGB)

[[6313   69]
 [ 267  209]]


Read the out-of-sample data (Testing Data)

In [14]:
new_data=pd.read_csv("C:/Users/Deepu/Desktop/Predict criminal/criminal_test.csv")
X_new=new_data.iloc[:,:].values

Predict on the new data using the trained classifier

In [15]:
new_pred=grid_searchXGB.predict(X_new)

Export the predicted data to CSV and rearrange the columns

In [16]:
results = pd.DataFrame(new_pred)
results.columns = ["Criminal"]
results['PERID']=new_data['PERID']
ColumnTitles=['PERID','Criminal']
results=results.reindex(columns=ColumnTitles)
results.to_csv("submission.csv",index=False)